It happens all the time: someone gives you data containing malformed strings,
Python, lists and missing data. How do you tidy it up so you can get on with the
analysis?
Take this monstrosity as the DataFrame to use in the following puzzles:
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN',
'londON_StockhOlm',
'Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
'12. Air France', '"Swiss Air"']})


In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN',
'londON_StockhOlm',
'Budapest_PaRis', 'Brussels_londOn'],
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )',
'12. Air France', '"Swiss Air"']})

In [3]:
data

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


1. Some values in the the FlightNumber column are missing. These numbers are
meant to increase by 10 with each row so 10055 and 10075 need to be put in
place. Fill in these missing numbers and make the column an integer column
(instead of a float column).

In [4]:
data['FlightNumber'].interpolate(inplace=True)

In [5]:
data[['FlightNumber']]=data['FlightNumber'].astype(int)
data

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055,[],<Air France> (12)
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075,[13],12. Air France
4,Brussels_londOn,10085,"[67, 32]","""Swiss Air"""


2. The From_To column would be better as two separate columns! Split each
string on the underscore delimiter _ to give a new temporary DataFrame with
the correct values. Assign the correct column names to this temporary
DataFrame.

In [6]:
new_data=data['From_To'].to_list()

In [7]:
new_data=[i.split('_') for i in new_data]
From_data=[i[0] for i in new_data]
To_data=[i[1] for i in new_data]

In [8]:
new_from_to=pd.DataFrame(data={'From':From_data,'To':To_data},columns=['From',"To"])
new_from_to

,From,To
0,LoNDon,paris
1,MAdrid,miLAN
2,londON,StockhOlm
3,Budapest,PaRis
4,Brussels,londOn


3. Notice how the capitalisation of the city names is all mixed up in this
temporary DataFrame. Standardise the strings so that only the first letter is
uppercase (e.g. "londON" should become "London".)

In [9]:
new_from_to['From']=[i.capitalize() for i in new_from_to['From']]
new_from_to['To']=[i.capitalize() for i in new_from_to['To']]
new_from_to

,From,To
0,London,Paris
1,Madrid,Milan
2,London,Stockholm
3,Budapest,Paris
4,Brussels,London


4. Delete the From_To column from df and attach the temporary DataFrame
from the previous questions.

In [10]:
data.drop(labels=['From_To'],axis=1,inplace=True)

In [11]:
data=pd.concat([new_from_to,data],axis=1,sort=False)
data

,From,To,FlightNumber,RecentDelays,Airline
0,London,Paris,10045,"[23, 47]",KLM(!)
1,Madrid,Milan,10055,[],<Air France> (12)
2,London,Stockholm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest,Paris,10075,[13],12. Air France
4,Brussels,London,10085,"[67, 32]","""Swiss Air"""


5. In the RecentDelays column, the values have been entered into the
DataFrame as a list. We would like each first value in its own column, each 
second value in its own column, and so on. If there isn't an Nth value, the value
should be NaN.

In [12]:
recentdelaysList=data['RecentDelays'].to_list()

In [13]:
delayds=pd.DataFrame(data=recentdelaysList,columns=['delay1','delay2','delay3'])

In [14]:
delayds

,delay1,delay2,delay3
0,23.0,47.0,NaN
1,NaN,NaN,NaN
2,24.0,43.0,87.0
3,13.0,NaN,NaN
4,67.0,32.0,NaN


Expand the Series of lists into a DataFrame named delays, rename the columns
delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df
with delays.

In [15]:
data.insert(loc=3,column="delay_1",value=delayds['delay1'])
data.insert(loc=4,column="delay_2",value=delayds['delay2'])
data.insert(loc=5,column="delay_3",value=delayds['delay3'])

In [16]:
data

,From,To,FlightNumber,delay_1,delay_2,delay_3,RecentDelays,Airline
0,London,Paris,10045,23.0,47.0,NaN,"[23, 47]",KLM(!)
1,Madrid,Milan,10055,NaN,NaN,NaN,[],<Air France> (12)
2,London,Stockholm,10065,24.0,43.0,87.0,"[24, 43, 87]",(British Airways. )
3,Budapest,Paris,10075,13.0,NaN,NaN,[13],12. Air France
4,Brussels,London,10085,67.0,32.0,NaN,"[67, 32]","""Swiss Air"""


In [17]:
data.drop(labels=['RecentDelays'],axis=1,inplace=True)

In [18]:
data

,From,To,FlightNumber,delay_1,delay_2,delay_3,Airline
0,London,Paris,10045,23.0,47.0,NaN,KLM(!)
1,Madrid,Milan,10055,NaN,NaN,NaN,<Air France> (12)
2,London,Stockholm,10065,24.0,43.0,87.0,(British Airways. )
3,Budapest,Paris,10075,13.0,NaN,NaN,12. Air France
4,Brussels,London,10085,67.0,32.0,NaN,"""Swiss Air"""
